In [2]:
import simpy
import numpy as np

# -----------------------------
# KONFIGURATION
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) # 3 Jahre Simulation
SEED = 42

# 1. NACHFRAGE
TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR 

# 2. KAPAZITÄT
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 19
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME 

# ANPASSUNG (KORRIGIERT):
# Mit der korrigierten Zeitschleife (siehe unten) ist das System nun dauerhaft belastet.
# Ein Faktor von 0.40 (40%) sorgt für eine Auslastung nah an 100%, 
# was zu realistischen Staus führt, ohne dass die Wartezeit ins Unendliche wächst.
EFFICIENCY_FACTOR = 0.40 
REAL_CAPACITY = int(THEORETICAL_CAPACITY * EFFICIENCY_FACTOR)

# 3. ZEIT-PARAMETER
SESSIONS_REQUIRED = 24  
WORKING_WEEKS = 42  # Abzug von Urlaub/Krankheit
TIME_STRETCH = 52 / WORKING_WEEKS 
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH # ca. 30 Wochen

# Wir starten mit voller Last
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005 

stats_wait_times = []

# -----------------------------
# PROZESSLOGIK
# -----------------------------

def patient_process(env, name, therapy_slots, dropped_counter):
    t_ankunft = env.now
    if np.random.random() > P_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    with therapy_slots.request() as req:
        yield req 
        stats_wait_times.append(env.now - t_ankunft)
        yield env.timeout(REAL_DURATION_WEEKS)

def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))

def setup(env, capacity, rng):
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))
        
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # KORREKTUR: Generator läuft nun über die volle Simulationsdauer!
    # (Vorher stand hier WEEKS_PER_YEAR, was falsch war)
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN & MEDIAN-ANALYSE
# -----------------------------
if __name__ == "__main__":
    print(f"--- SIMULATION (MIT MEDIAN & KORRIGIERTEM LOOP) ---")
    print(f"Laufzeit: {SIMULATION_DURATION} Wochen")
    print(f"Effizienz-Faktor: {EFFICIENCY_FACTOR}")
    print(f"Kapazität: {REAL_CAPACITY} Slots")
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n" + "="*60)
    print(f"       STATISTIK (MEDIAN VS. MITTELWERT)       ")
    print(f"="*60)
    
    if n_started > 0:
        # Mittelwert
        mean_weeks = np.mean(waits)
        mean_days = mean_weeks * 7
        
        # Median (50-Prozent-Hürde)
        median_weeks = np.median(waits)
        median_days = median_weeks * 7
        
        # P90 (Risiko)
        p90_weeks = np.quantile(waits, 0.90)
        p90_days = p90_weeks * 7

        print(f"Anzahl Patienten: {n_started}")
        print(f"-"*60)
        # Hier sehen Sie den Unterschied direkt:
        print(f"MITTELWERT (Mean):       {mean_days:6.1f} Tage")
        print(f"MEDIAN (Typischer Fall): {median_days:6.1f} Tage")
        print(f"-"*60)
        print(f"90% warten kürzer als:   {p90_days:6.1f} Tage")
        print(f"Maximale Wartezeit:      {np.max(waits)*7:6.1f} Tage")
        
    else:
        print("Keine Daten.")

--- SIMULATION (MIT MEDIAN & KORRIGIERTEM LOOP) ---
Laufzeit: 156 Wochen
Effizienz-Faktor: 0.4
Kapazität: 46440 Slots

       STATISTIK (MEDIAN VS. MITTELWERT)       
Anzahl Patienten: 243820
------------------------------------------------------------
MITTELWERT (Mean):        118.8 Tage
MEDIAN (Typischer Fall):  101.3 Tage
------------------------------------------------------------
90% warten kürzer als:    262.5 Tage
Maximale Wartezeit:       310.4 Tage
